In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("data/", one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [3]:
def init_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

In [4]:
def init_bias(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

In [5]:
def conv_2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")

In [6]:
def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")    

In [7]:
def conv_layer(X_input, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv_2d(X_input, W) + b)

In [8]:
def fully_connected_layer(L_input, size):
    input_size = int(L_input.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(L_input, W) + b 

In [9]:
X_train = tf.placeholder(tf.float32, shape=[None, 784])
X_train_reshaped = tf.reshape(X_train, [-1, 28, 28, 1])

y_train = tf.placeholder(tf.float32, shape=[None, 10])

In [10]:
conv_1 = conv_layer(X_train_reshaped, [6, 6, 1, 32])
conv_1_pooling = max_pool_2by2(conv_1)

In [11]:
conv_2 = conv_layer(conv_1_pooling, [6, 6, 32, 64])
conv_2_pooling = max_pool_2by2(conv_2)

In [12]:
conv_2_flat = tf.reshape(conv_2_pooling, [-1, 7*7*64])
fully_connected_1 = tf.nn.relu(fully_connected_layer(conv_2_flat, 1024)) 

In [13]:
dropout_hold_probability = tf.placeholder(tf.float32)
dropout = tf.nn.dropout(fully_connected_1, keep_prob=dropout_hold_probability)

In [14]:
y_model = fully_connected_layer(dropout, 10)

In [15]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_train, logits=y_model))

In [16]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0003)
train = optimizer.minimize(loss)

In [17]:
init = tf.global_variables_initializer()

In [18]:
EPOCHS = 10000

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)

with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as session:
    session.run(init)
    
    for epoch in range(EPOCHS+1):
        X_batch, y_batch = mnist.train.next_batch(16)
        
        session.run(train, feed_dict={X_train:X_batch, y_train:y_batch, dropout_hold_probability:0.5})
        
        if epoch % 2000 == 0:
            print('EPOCH: {}'.format(epoch))
            
            matches = tf.equal(tf.argmax(y_model, 1),tf.argmax(y_train, 1))
            accuracy = tf.reduce_mean(tf.cast(matches, tf.float32))

            print(session.run(accuracy, feed_dict={X_train:mnist.test.images[:5000],y_train:mnist.test.labels[:5000],
                                                dropout_hold_probability:1.0}))
            print('\n')

EPOCH: 0
0.0876


EPOCH: 2000
0.9684


EPOCH: 4000
0.9724


EPOCH: 6000
0.9788


EPOCH: 8000
0.9808


EPOCH: 10000
0.982


